In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
import numpy as np 
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt  
import seaborn as sns 
color = sns.color_palette()


In [2]:
application_train = pd.read_csv('application_train.csv')
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
previous_application = pd.read_csv('previous_application.csv')
installments_payments = pd.read_csv('installments_payments.csv')
credit_card_balance = pd.read_csv('credit_card_balance.csv')
bureau = pd.read_csv('bureau.csv')
application_test = pd.read_csv('application_test.csv')

In [3]:

bureau['DAYS_CREDIT_ENDDATE'][bureau['DAYS_CREDIT_ENDDATE'] < -40000] = np.nan
bureau['DAYS_CREDIT_UPDATE'][bureau['DAYS_CREDIT_UPDATE'] < -40000] = np.nan
bureau['DAYS_ENDDATE_FACT'][bureau['DAYS_ENDDATE_FACT'] < -40000] = np.nan
credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'][credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'] < 0] = np.nan
credit_card_balance['AMT_DRAWINGS_CURRENT'][credit_card_balance['AMT_DRAWINGS_CURRENT'] < 0] = np.nan
POS_CASH_balance['pos_cash_paid_late'] = (POS_CASH_balance['SK_DPD'] > 0).astype(int)
POS_CASH_balance['pos_cash_paid_late_with_tolerance'] = (POS_CASH_balance['SK_DPD_DEF'] > 0).astype(int)
installments_payments['instalment_paid_late_in_days'] = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT'] 
installments_payments['instalment_paid_late'] = (installments_payments['instalment_paid_late_in_days'] > 0).astype(int)
installments_payments['instalment_paid_over_amount'] = installments_payments['AMT_PAYMENT'] - installments_payments['AMT_INSTALMENT']
installments_payments['instalment_paid_over'] = (installments_payments['instalment_paid_over_amount'] > 0).astype(int)

In [4]:
all_data = application_train.append(application_test, ignore_index=True)


mask = all_data.DAYS_EMPLOYED >0
column_name = "DAYS_EMPLOYED"
all_data.loc[mask, column_name] = None
all_data['DAYS_EMPLOYED'] = all_data['DAYS_EMPLOYED'].abs()
all_data['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)
all_data['CODE_GENDER'].replace('XNA', np.nan, inplace=True)
all_data['NAME_FAMILY_STATUS'].replace('Unknown', np.nan, inplace=True)
all_data['ORGANIZATION_TYPE'].replace('XNA', np.nan, inplace=True)
all_data['DAYS_LAST_PHONE_CHANGE'] = all_data['DAYS_LAST_PHONE_CHANGE'].abs()
all_data['DAYS_ID_PUBLISH'] = all_data['DAYS_ID_PUBLISH'].abs()
all_data['DAYS_REGISTRATION'] = all_data['DAYS_REGISTRATION'].abs()
all_data['DAYS_BIRTH'] = all_data['DAYS_BIRTH'].abs()


In [5]:
all_data['NEW_CREDIT_TO_ANNUITY_RATIO'] = all_data['AMT_CREDIT'] / all_data['AMT_ANNUITY']
all_data['NEW_CREDIT_TO_GOODS_RATIO'] = all_data['AMT_CREDIT'] / all_data['AMT_GOODS_PRICE']
all_data['NEW_INC_PER_CHLD'] = all_data['AMT_INCOME_TOTAL'] / (1 + all_data['CNT_CHILDREN'])
all_data['NEW_INC_PER__FAM_MEMBERS'] = all_data['AMT_INCOME_TOTAL'] / (1 + all_data['CNT_FAM_MEMBERS'])
all_data['NEW_EMPLOY_TO_BIRTH_RATIO'] = all_data['DAYS_EMPLOYED'] / all_data['DAYS_BIRTH']
all_data['NEW_ANNUITY_TO_INCOME_RATIO'] = all_data['AMT_ANNUITY'] / (1 + all_data['AMT_INCOME_TOTAL'])
all_data['New_children_ratio'] = all_data['CNT_CHILDREN'] / all_data['CNT_FAM_MEMBERS'] 
all_data['days_employed_percentage'] = all_data['DAYS_EMPLOYED'] / all_data['DAYS_BIRTH']
all_data['NEW_EXT_SOURCES_add'] =  all_data['EXT_SOURCE_1'] *2+ all_data['EXT_SOURCE_2'] *3+ all_data['EXT_SOURCE_3'] *4
all_data['NEW_SCORES_STD'] = all_data[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
all_data['NEW_SCORES_STD'] = all_data['NEW_SCORES_STD'].fillna(all_data['NEW_SCORES_STD'].mean())
all_data['NEW_CAR_TO_BIRTH_RATIO'] = all_data['OWN_CAR_AGE'] / all_data['DAYS_BIRTH']
all_data['NEW_CAR_TO_EMPLOY_RATIO'] = all_data['OWN_CAR_AGE'] / all_data['DAYS_EMPLOYED']
all_data['NEW_PHONE_TO_BIRTH_RATIO'] = all_data['DAYS_LAST_PHONE_CHANGE'] / all_data['DAYS_BIRTH']
all_data['NEW_PHONE_TO_EMPLOY_RATIO'] = all_data['DAYS_LAST_PHONE_CHANGE'] / all_data['DAYS_EMPLOYED']
all_data['NEW_CREDIT_TO_INCOME_RATIO'] = all_data['AMT_CREDIT'] / all_data['AMT_INCOME_TOTAL']
docs = [_f for _f in all_data.columns if 'FLAG_DOC' in _f]
live = [_f for _f in all_data.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
all_data['NEW_DOC_IND_KURT'] = all_data[docs].kurtosis(axis=1)
all_data['NEW_LIVE_IND_SUM'] = all_data[live].sum(axis=1)
inc_by_org = all_data[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
all_data['NEW_INC_BY_ORG'] = all_data['ORGANIZATION_TYPE'].map(inc_by_org)

all_data['NEW_FLAG_DOC_SUM'] = all_data[['FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4',
    'FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7',
    'FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 
    'FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13',
    'FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
    'FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20','FLAG_DOCUMENT_21']].sum(axis=1)
all_data['NEW_FLAG_CONTACT_SUM'] = all_data[['FLAG_PHONE', 'FLAG_EMAIL', 'FLAG_CONT_MOBILE','FLAG_WORK_PHONE','FLAG_EMP_PHONE','FLAG_MOBIL']].sum(axis=1)
for function_name in ['min', 'max', 'sum', 'mean', 'nanmedian']:
    all_data['external_sources_{}'.format(function_name)] = eval('np.{}'.format(function_name))(
        all_data[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

In [6]:
del docs
del live
del inc_by_org

In [7]:
all_data['new_cnt_non_child'] = all_data['CNT_FAM_MEMBERS'] - all_data['CNT_CHILDREN']
all_data['new_child_to_non_child_ratio'] = all_data['CNT_CHILDREN'] / all_data['new_cnt_non_child']
all_data['new_income_per_non_child'] = all_data['AMT_INCOME_TOTAL'] / all_data['new_cnt_non_child']
all_data['new_credit_per_person'] = all_data['AMT_CREDIT'] / all_data['CNT_FAM_MEMBERS']
all_data['new_credit_per_child'] = all_data['AMT_CREDIT'] / (1 + all_data['CNT_CHILDREN'])
all_data['new_credit_per_non_child'] = all_data['AMT_CREDIT'] / all_data['new_cnt_non_child']

In [8]:
# kaihua new feature
grp = bureau[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'BUREAU_LOAN_COUNT'})
all_data = all_data.merge(grp, on = ['SK_ID_CURR'], how = 'left')
bureau['CREDIT_ACTIVE_BINARY'] = bureau['CREDIT_ACTIVE']
def f(x):                 # Create a new dummy column for whether CREDIT is ACTIVE OR CLOED 
    if x == 'Closed':
        y = 0
    else:
        y = 1    
    return y

bureau['CREDIT_ACTIVE_BINARY'] = bureau.apply(lambda x: f(x.CREDIT_ACTIVE), axis = 1)
bureau['CREDIT_ACTIVE_BINARY'].head(5)
grp = bureau[['SK_ID_CURR', 'CREDIT_ACTIVE_BINARY']].groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].sum().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'BUREAU_active_COUNT'})
all_data = all_data.merge(grp, on = ['SK_ID_CURR'], how = 'left')
grp = bureau.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'B_ACTIVE_LOANS_PERCENTAGE'})
all_data = all_data.merge(grp, on = ['SK_ID_CURR'], how = 'left')


In [9]:
del bureau['CREDIT_ACTIVE_BINARY']

In [10]:
# Groupby each Customer and Sort values of DAYS_CREDIT in ascending order
grp =bureau[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)#rename(index = str, columns = {'DAYS_CREDIT': 'DAYS_CREDIT_DIFF'})
grp1.head(5)

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT
0,100001,5896635,-49
1,100001,5896636,-320
2,100001,5896634,-559
3,100001,5896630,-857
4,100001,5896632,-879


In [11]:
# Calculate Difference between the number of Days 
grp1['DAYS_CREDIT_POSITIVE'] = -1*grp1['DAYS_CREDIT']
grp1['DAYS_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT_POSITIVE'].diff()
grp1['DAYS_DIFF'] = grp1['DAYS_DIFF'].fillna(0).astype('uint32')
grp1.head(5)

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,DAYS_CREDIT_POSITIVE,DAYS_DIFF
0,100001,5896635,-49,49,0
1,100001,5896636,-320,320,271
2,100001,5896634,-559,559,239
3,100001,5896630,-857,857,298
4,100001,5896632,-879,879,22


In [12]:
import gc
del grp1['DAYS_CREDIT_POSITIVE'], grp1['DAYS_CREDIT'], grp1['SK_ID_CURR']
gc.collect()

bureau = bureau.merge(grp1, on = ['SK_ID_BUREAU'], how = 'left')
bureau.shape

(1716428, 18)

In [13]:
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)
bureau['AMT_CREDIT_SUM'] = bureau['AMT_CREDIT_SUM'].fillna(0)

grp1 = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
grp2 = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM': 'TOTAL_CUSTOMER_CREDIT'})

all_data = all_data.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
all_data = all_data.merge(grp2, on = ['SK_ID_CURR'], how = 'left')
all_data['B_DEBT_CREDIT_RATIO'] = all_data['TOTAL_CUSTOMER_DEBT']/all_data['TOTAL_CUSTOMER_CREDIT']
del all_data['TOTAL_CUSTOMER_DEBT'], all_data['TOTAL_CUSTOMER_CREDIT']
gc.collect()


71

In [14]:
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)
bureau['AMT_CREDIT_SUM_OVERDUE'] = bureau['AMT_CREDIT_SUM_OVERDUE'].fillna(0)

grp1 = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
grp2 = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_OVERDUE']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_OVERDUE': 'TOTAL_CUSTOMER_OVERDUE'})

all_data = all_data.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
all_data = all_data.merge(grp2, on = ['SK_ID_CURR'], how = 'left')
del grp1, grp2
gc.collect()

all_data['B_OVERDUE_DEBT_RATIO'] = all_data['TOTAL_CUSTOMER_OVERDUE']/all_data['TOTAL_CUSTOMER_DEBT']

del all_data['TOTAL_CUSTOMER_OVERDUE'], all_data['TOTAL_CUSTOMER_DEBT']
gc.collect()
all_data.shape

(356255, 158)

In [15]:
# No of Loans per customer 
grp = credit_card_balance.groupby(by = ['SK_ID_CURR'])['SK_ID_PREV'].nunique().reset_index().rename(index = str, columns = {'SK_ID_PREV': 'C_NO_LOANS'})
all_data = all_data.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

36

In [16]:
grp = credit_card_balance.groupby(by = ['SK_ID_CURR', 'SK_ID_PREV'])['CNT_INSTALMENT_MATURE_CUM'].max().reset_index().rename(index = str, columns = {'CNT_INSTALMENT_MATURE_CUM': 'NO_INSTALMENTS'})
grp1 = grp.groupby(by = ['SK_ID_CURR'])['NO_INSTALMENTS'].sum().reset_index().rename(index = str, columns = {'NO_INSTALMENTS': 'C_TOTAL_INSTALMENTS'})
all_data = all_data.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
del grp, grp1
gc.collect()

21

In [17]:
all_data['C_NO_LOANS'].fillna(0).isnull().sum()

0

In [18]:
# Average Number of installments paid per loan 

all_data['INSTALLMENTS_PER_LOAN'] = (all_data['C_TOTAL_INSTALMENTS'].fillna(0)/(1+all_data['C_NO_LOANS'].fillna(0))).astype('uint32')
del all_data['C_TOTAL_INSTALMENTS']
del all_data['C_NO_LOANS']
gc.collect()

all_data.shape

(356255, 159)

In [19]:
def f(DPD):
    x = DPD.tolist()
    c = 0
    for i,j in enumerate(x):
        if j != 0:
            c += 1
    return c 

grp = credit_card_balance.groupby(by = ['SK_ID_CURR', 'SK_ID_PREV']).apply(lambda x: f(x.SK_DPD)).reset_index().rename(index = str, columns = {0: 'NO_DPD'})
grp1 = grp.groupby(by = ['SK_ID_CURR'])['NO_DPD'].mean().reset_index().rename(index = str, columns = {'NO_DPD' : 'DPD_COUNT'})

all_data = all_data.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
del grp1
del grp 
gc.collect()

all_data.shape

(356255, 160)

In [20]:
grp = credit_card_balance.groupby(by= ['SK_ID_CURR'])['SK_DPD'].mean().reset_index().rename(index = str, columns = {'SK_DPD': 'AVG_DPD'})
all_data = all_data.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

all_data.shape

(356255, 161)

In [21]:
def f(min_pay, total_pay):
    
    M = min_pay.tolist()
    T = total_pay.tolist()
    P = len(M)
    c = 0 
    # Find the count of transactions when Payment made is less than Minimum Payment 
    for i in range(len(M)):
        if T[i] < M[i]:
            c += 1  
    return (100*c)/P

grp = credit_card_balance.groupby(by = ['SK_ID_CURR']).apply(lambda x: f(x.AMT_INST_MIN_REGULARITY, x.AMT_PAYMENT_CURRENT)).reset_index().rename(index = str, columns = { 0 : 'PERCENTAGE_MISSED_PAYMENTS'})
all_data = all_data.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp 
gc.collect()

all_data.shape

(356255, 162)

In [22]:
grp = credit_card_balance.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_ATM_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_ATM_CURRENT' : 'DRAWINGS_ATM'})
all_data = all_data.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = credit_card_balance.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_CURRENT' : 'DRAWINGS_TOTAL'})
all_data = all_data.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

all_data['CASH_CARD_RATIO1'] = (all_data['DRAWINGS_ATM']/all_data['DRAWINGS_TOTAL'])*100
del all_data['DRAWINGS_ATM']
del all_data['DRAWINGS_TOTAL']
gc.collect()
all_data.shape

(356255, 163)

In [23]:
grp = credit_card_balance.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_CURRENT' : 'TOTAL_DRAWINGS'})
all_data = all_data.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

grp = credit_card_balance.groupby(by = ['SK_ID_CURR'])['CNT_DRAWINGS_CURRENT'].sum().reset_index().rename(index = str, columns = {'CNT_DRAWINGS_CURRENT' : 'NO_DRAWINGS'})
all_data = all_data.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

all_data['DRAWINGS_RATIO1'] = (all_data['TOTAL_DRAWINGS']/all_data['NO_DRAWINGS'])*100
del all_data['TOTAL_DRAWINGS']
del all_data['NO_DRAWINGS']
gc.collect()

all_data.shape

(356255, 164)

In [31]:
del application_test

In [25]:
from tqdm import tqdm_notebook as tqdm

In [26]:
AGGREGATION_RECIPIES = [
    (['CODE_GENDER', 'NAME_EDUCATION_TYPE'], [('AMT_ANNUITY', 'max'),
                                              ('AMT_CREDIT', 'max'),
                                              ('EXT_SOURCE_1', 'mean'),
                                              ('EXT_SOURCE_2', 'mean'),
                                              ('OWN_CAR_AGE', 'max'),
                                              ('OWN_CAR_AGE', 'sum')]),
    (['CODE_GENDER', 'ORGANIZATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                                            ('AMT_INCOME_TOTAL', 'mean'),
                                            ('DAYS_REGISTRATION', 'mean'),
                                            ('EXT_SOURCE_1', 'mean')]),
    (['CODE_GENDER', 'REG_CITY_NOT_WORK_CITY'], [('AMT_ANNUITY', 'mean'),
                                                 ('CNT_CHILDREN', 'mean'),
                                                 ('DAYS_ID_PUBLISH', 'mean')]),
    (['CODE_GENDER', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('EXT_SOURCE_1', 'mean'),
                                                                                           ('EXT_SOURCE_2', 'mean')]),
    (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE'], [('AMT_CREDIT', 'mean'),
                                                  ('AMT_REQ_CREDIT_BUREAU_YEAR', 'mean'),
                                                  ('APARTMENTS_AVG', 'mean'),
                                                  ('BASEMENTAREA_AVG', 'mean'),
                                                  ('EXT_SOURCE_1', 'mean'),
                                                  ('EXT_SOURCE_2', 'mean'),
                                                  ('EXT_SOURCE_3', 'mean'),
                                                  ('NONLIVINGAREA_AVG', 'mean'),
                                                  ('OWN_CAR_AGE', 'mean'),
                                                  ('YEARS_BUILD_AVG', 'mean')]),
    (['NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'REG_CITY_NOT_WORK_CITY'], [('ELEVATORS_AVG', 'mean'),
                                                                            ('EXT_SOURCE_1', 'mean')]),
    (['OCCUPATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                           ('CNT_CHILDREN', 'mean'),
                           ('CNT_FAM_MEMBERS', 'mean'),
                           ('DAYS_BIRTH', 'mean'),
                           ('DAYS_EMPLOYED', 'mean'),
                           ('DAYS_ID_PUBLISH', 'mean'),
                           ('DAYS_REGISTRATION', 'mean'),
                           ('EXT_SOURCE_1', 'mean'),
                           ('EXT_SOURCE_2', 'mean'),
                           ('EXT_SOURCE_3', 'mean')]),
]

In [27]:
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(AGGREGATION_RECIPIES):
    group_object = all_data.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        all_data = all_data.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [28]:
diff_feature_names = []
for groupby_cols, specs in tqdm(AGGREGATION_RECIPIES):
    for select, agg in tqdm(specs):
        if agg in ['mean','median','max','min']:
            groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
            diff_name = '{}_diff'.format(groupby_aggregate_name)
            abs_diff_name = '{}_abs_diff'.format(groupby_aggregate_name)

            all_data[diff_name] = all_data[select] - all_data[groupby_aggregate_name] 
            all_data[abs_diff_name] = np.abs(all_data[select] - all_data[groupby_aggregate_name]) 

            diff_feature_names.append(diff_name)
            diff_feature_names.append(abs_diff_name)

In [29]:
all_data['retirement_age'] = (all_data['DAYS_BIRTH'] >20000).astype(int)
all_data['long_employment'] = (all_data['DAYS_EMPLOYED']>2000).astype(int)

In [30]:
all_data.shape

(356255, 275)

In [30]:
previous_application.columns.values


array(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT',
       'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT',
       'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT',
       'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED',
       'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION',
       'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE',
       'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO',
       'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'SELLERPLACE_AREA',
       'NAME_SELLER_INDUSTRY', 'CNT_PAYMENT', 'NAME_YIELD_GROUP',
       'PRODUCT_COMBINATION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE',
       'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION',
       'NFLAG_INSURED_ON_APPROVAL'], dtype=object)

In [31]:
previous_application.select_dtypes(exclude=['object']).columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'AMT_ANNUITY', 'AMT_APPLICATION',
       'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA',
       'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE',
       'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION',
       'NFLAG_INSURED_ON_APPROVAL'], dtype=object)

In [32]:
AGGREGATION_RECIPIES = [
    (['WEEKDAY_APPR_PROCESS_START'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                               ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),(['NAME_GOODS_CATEGORY'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                               ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),
    
    (['CODE_REJECT_REASON'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                               ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),(['NAME_GOODS_CATEGORY'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                               ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),
    
    (['NFLAG_INSURED_ON_APPROVAL'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                               ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),
        (['NAME_TYPE_SUITE'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                               ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),        
    (['NAME_PRODUCT_TYPE'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                             ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),        
    (['CHANNEL_TYPE'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                        ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),        
    (['FLAG_LAST_APPL_PER_CONTRACT'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),
        (['NAME_CASH_LOAN_PURPOSE'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                                      ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),
     (['NAME_CLIENT_TYPE'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                               ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),
        (['NAME_YIELD_GROUP'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                                ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),        
    (['PRODUCT_COMBINATION'], [('AMT_ANNUITY', 'mean'),
                           ('AMT_APPLICATION', 'mean'),
                           ('AMT_CREDIT', 'mean'),
                           ('AMT_DOWN_PAYMENT', 'mean'),
                               ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),  
                            ('DAYS_TERMINATION', 'mean'),
                           ('AMT_GOODS_PRICE', 'mean'),
                           ('CNT_PAYMENT', 'mean'),
                            ('DAYS_DECISION', 'mean'),
                           ('HOUR_APPR_PROCESS_START', 'mean'),
                           ('RATE_DOWN_PAYMENT', 'mean')]),

]

In [33]:
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(AGGREGATION_RECIPIES):
    group_object = previous_application.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        previous_application = previous_application.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [34]:
previous_application['AMT_CREDIT_to_AMT_ANNUITY_ratio']=previous_application['AMT_CREDIT'] / previous_application['AMT_ANNUITY']
previous_application['AMT_CREDIT_to_AMT_GOODS_PRICE_ratio']=previous_application['AMT_CREDIT'] / previous_application['AMT_GOODS_PRICE']
previous_application['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
previous_application['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
previous_application['APP_CREDIT_PERC'] = previous_application['AMT_APPLICATION'] / previous_application['AMT_CREDIT']
#previous_application = pd.get_dummies(previous_application)

In [35]:
previous_application_agg_RECIPIES = []
for agg in [ 'mean','min', 'max', 'sum', 'var']:
    for select in [ 'AMT_ANNUITY', 'AMT_APPLICATION',
       'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA',
       'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE',
       'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION',
       'NFLAG_INSURED_ON_APPROVAL', 'AMT_CREDIT_to_AMT_ANNUITY_ratio',
       'AMT_CREDIT_to_AMT_GOODS_PRICE_ratio', 'APP_CREDIT_PERC'
                   ]:
        previous_application_agg_RECIPIES.append((select, agg))
previous_application_agg_RECIPIES = [(['SK_ID_CURR'], previous_application_agg_RECIPIES)]
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(previous_application_agg_RECIPIES):
    group_object = previous_application.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        all_data = all_data.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [36]:
previous_application_agg_RECIPIES = []
for agg in [ 'mean','max']:
    for select in ['WEEKDAY_APPR_PROCESS_START_mean_AMT_ANNUITY',
       'WEEKDAY_APPR_PROCESS_START_mean_AMT_APPLICATION',
       'WEEKDAY_APPR_PROCESS_START_mean_AMT_CREDIT',
       'WEEKDAY_APPR_PROCESS_START_mean_AMT_DOWN_PAYMENT',
       'WEEKDAY_APPR_PROCESS_START_mean_DAYS_LAST_DUE_1ST_VERSION',
       'WEEKDAY_APPR_PROCESS_START_mean_DAYS_TERMINATION',
       'WEEKDAY_APPR_PROCESS_START_mean_AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START_mean_CNT_PAYMENT',
       'WEEKDAY_APPR_PROCESS_START_mean_DAYS_DECISION',
       'WEEKDAY_APPR_PROCESS_START_mean_HOUR_APPR_PROCESS_START',
       'WEEKDAY_APPR_PROCESS_START_mean_RATE_DOWN_PAYMENT',
       'NAME_GOODS_CATEGORY_mean_AMT_ANNUITY_x',
       'NAME_GOODS_CATEGORY_mean_AMT_APPLICATION_x',
       'NAME_GOODS_CATEGORY_mean_AMT_CREDIT_x',
       'NAME_GOODS_CATEGORY_mean_AMT_DOWN_PAYMENT_x',
       'NAME_GOODS_CATEGORY_mean_DAYS_LAST_DUE_1ST_VERSION_x',
       'NAME_GOODS_CATEGORY_mean_DAYS_TERMINATION_x',
       'NAME_GOODS_CATEGORY_mean_AMT_GOODS_PRICE_x',
       'NAME_GOODS_CATEGORY_mean_CNT_PAYMENT_x',
       'NAME_GOODS_CATEGORY_mean_DAYS_DECISION_x',
       'NAME_GOODS_CATEGORY_mean_HOUR_APPR_PROCESS_START_x',
       'NAME_GOODS_CATEGORY_mean_RATE_DOWN_PAYMENT_x',
       'CODE_REJECT_REASON_mean_AMT_ANNUITY',
       'CODE_REJECT_REASON_mean_AMT_APPLICATION',
       'CODE_REJECT_REASON_mean_AMT_CREDIT',
       'CODE_REJECT_REASON_mean_AMT_DOWN_PAYMENT',
       'CODE_REJECT_REASON_mean_DAYS_LAST_DUE_1ST_VERSION',
       'CODE_REJECT_REASON_mean_DAYS_TERMINATION',
       'CODE_REJECT_REASON_mean_AMT_GOODS_PRICE',
       'CODE_REJECT_REASON_mean_CNT_PAYMENT',
       'CODE_REJECT_REASON_mean_DAYS_DECISION',
       'CODE_REJECT_REASON_mean_HOUR_APPR_PROCESS_START',
       'CODE_REJECT_REASON_mean_RATE_DOWN_PAYMENT',
       'NAME_GOODS_CATEGORY_mean_AMT_ANNUITY_y',
       'NAME_GOODS_CATEGORY_mean_AMT_APPLICATION_y',
       'NAME_GOODS_CATEGORY_mean_AMT_CREDIT_y',
       'NAME_GOODS_CATEGORY_mean_AMT_DOWN_PAYMENT_y',
       'NAME_GOODS_CATEGORY_mean_DAYS_LAST_DUE_1ST_VERSION_y',
       'NAME_GOODS_CATEGORY_mean_DAYS_TERMINATION_y',
       'NAME_GOODS_CATEGORY_mean_AMT_GOODS_PRICE_y',
       'NAME_GOODS_CATEGORY_mean_CNT_PAYMENT_y',
       'NAME_GOODS_CATEGORY_mean_DAYS_DECISION_y',
       'NAME_GOODS_CATEGORY_mean_HOUR_APPR_PROCESS_START_y',
       'NAME_GOODS_CATEGORY_mean_RATE_DOWN_PAYMENT_y',
       'NFLAG_INSURED_ON_APPROVAL_mean_AMT_ANNUITY',
       'NFLAG_INSURED_ON_APPROVAL_mean_AMT_APPLICATION',
       'NFLAG_INSURED_ON_APPROVAL_mean_AMT_CREDIT',
       'NFLAG_INSURED_ON_APPROVAL_mean_AMT_DOWN_PAYMENT',
       'NFLAG_INSURED_ON_APPROVAL_mean_DAYS_LAST_DUE_1ST_VERSION',
       'NFLAG_INSURED_ON_APPROVAL_mean_DAYS_TERMINATION',
       'NFLAG_INSURED_ON_APPROVAL_mean_AMT_GOODS_PRICE',
       'NFLAG_INSURED_ON_APPROVAL_mean_CNT_PAYMENT',
       'NFLAG_INSURED_ON_APPROVAL_mean_DAYS_DECISION',
       'NFLAG_INSURED_ON_APPROVAL_mean_HOUR_APPR_PROCESS_START',
       'NFLAG_INSURED_ON_APPROVAL_mean_RATE_DOWN_PAYMENT',
       'NAME_TYPE_SUITE_mean_AMT_ANNUITY',
       'NAME_TYPE_SUITE_mean_AMT_APPLICATION',
       'NAME_TYPE_SUITE_mean_AMT_CREDIT',
       'NAME_TYPE_SUITE_mean_AMT_DOWN_PAYMENT',
       'NAME_TYPE_SUITE_mean_AMT_GOODS_PRICE',
       'NAME_TYPE_SUITE_mean_CNT_PAYMENT',
       'NAME_TYPE_SUITE_mean_DAYS_DECISION',
       'NAME_TYPE_SUITE_mean_DAYS_LAST_DUE_1ST_VERSION',
       'NAME_TYPE_SUITE_mean_DAYS_TERMINATION',
       'NAME_TYPE_SUITE_mean_HOUR_APPR_PROCESS_START',
       'NAME_TYPE_SUITE_mean_RATE_DOWN_PAYMENT',
       'NAME_PRODUCT_TYPE_mean_AMT_ANNUITY',
       'NAME_PRODUCT_TYPE_mean_AMT_APPLICATION',
       'NAME_PRODUCT_TYPE_mean_AMT_CREDIT',
       'NAME_PRODUCT_TYPE_mean_AMT_DOWN_PAYMENT',
       'NAME_PRODUCT_TYPE_mean_AMT_GOODS_PRICE',
       'NAME_PRODUCT_TYPE_mean_CNT_PAYMENT',
       'NAME_PRODUCT_TYPE_mean_DAYS_DECISION',
       'NAME_PRODUCT_TYPE_mean_DAYS_LAST_DUE_1ST_VERSION',
       'NAME_PRODUCT_TYPE_mean_DAYS_TERMINATION',
       'NAME_PRODUCT_TYPE_mean_HOUR_APPR_PROCESS_START',
       'NAME_PRODUCT_TYPE_mean_RATE_DOWN_PAYMENT',
       'CHANNEL_TYPE_mean_AMT_ANNUITY',
       'CHANNEL_TYPE_mean_AMT_APPLICATION',
       'CHANNEL_TYPE_mean_AMT_CREDIT',
       'CHANNEL_TYPE_mean_AMT_DOWN_PAYMENT',
       'CHANNEL_TYPE_mean_AMT_GOODS_PRICE',
       'CHANNEL_TYPE_mean_DAYS_LAST_DUE_1ST_VERSION',
       'CHANNEL_TYPE_mean_DAYS_TERMINATION',
       'CHANNEL_TYPE_mean_CNT_PAYMENT', 'CHANNEL_TYPE_mean_DAYS_DECISION',
       'CHANNEL_TYPE_mean_HOUR_APPR_PROCESS_START',
       'CHANNEL_TYPE_mean_RATE_DOWN_PAYMENT',
       'FLAG_LAST_APPL_PER_CONTRACT_mean_AMT_ANNUITY',
       'FLAG_LAST_APPL_PER_CONTRACT_mean_AMT_APPLICATION',
       'FLAG_LAST_APPL_PER_CONTRACT_mean_AMT_CREDIT',
       'FLAG_LAST_APPL_PER_CONTRACT_mean_AMT_DOWN_PAYMENT',
       'FLAG_LAST_APPL_PER_CONTRACT_mean_AMT_GOODS_PRICE',
       'FLAG_LAST_APPL_PER_CONTRACT_mean_CNT_PAYMENT',
       'FLAG_LAST_APPL_PER_CONTRACT_mean_DAYS_LAST_DUE_1ST_VERSION',
       'FLAG_LAST_APPL_PER_CONTRACT_mean_DAYS_TERMINATION',
       'FLAG_LAST_APPL_PER_CONTRACT_mean_DAYS_DECISION',
       'FLAG_LAST_APPL_PER_CONTRACT_mean_HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT_mean_RATE_DOWN_PAYMENT',
       'NAME_CASH_LOAN_PURPOSE_mean_AMT_ANNUITY',
       'NAME_CASH_LOAN_PURPOSE_mean_AMT_APPLICATION',
       'NAME_CASH_LOAN_PURPOSE_mean_AMT_CREDIT',
       'NAME_CASH_LOAN_PURPOSE_mean_AMT_DOWN_PAYMENT',
       'NAME_CASH_LOAN_PURPOSE_mean_AMT_GOODS_PRICE',
       'NAME_CASH_LOAN_PURPOSE_mean_CNT_PAYMENT',
       'NAME_CASH_LOAN_PURPOSE_mean_DAYS_DECISION',
       'NAME_CASH_LOAN_PURPOSE_mean_DAYS_LAST_DUE_1ST_VERSION',
       'NAME_CASH_LOAN_PURPOSE_mean_DAYS_TERMINATION',
       'NAME_CASH_LOAN_PURPOSE_mean_HOUR_APPR_PROCESS_START',
       'NAME_CASH_LOAN_PURPOSE_mean_RATE_DOWN_PAYMENT',
       'NAME_CLIENT_TYPE_mean_AMT_ANNUITY',
       'NAME_CLIENT_TYPE_mean_AMT_APPLICATION',
       'NAME_CLIENT_TYPE_mean_AMT_CREDIT',
       'NAME_CLIENT_TYPE_mean_AMT_DOWN_PAYMENT',
       'NAME_CLIENT_TYPE_mean_DAYS_LAST_DUE_1ST_VERSION',
       'NAME_CLIENT_TYPE_mean_DAYS_TERMINATION',
       'NAME_CLIENT_TYPE_mean_AMT_GOODS_PRICE',
       'NAME_CLIENT_TYPE_mean_CNT_PAYMENT',
       'NAME_CLIENT_TYPE_mean_DAYS_DECISION',
       'NAME_CLIENT_TYPE_mean_HOUR_APPR_PROCESS_START',
       'NAME_CLIENT_TYPE_mean_RATE_DOWN_PAYMENT',
       'NAME_YIELD_GROUP_mean_AMT_ANNUITY',
       'NAME_YIELD_GROUP_mean_AMT_APPLICATION',
       'NAME_YIELD_GROUP_mean_AMT_CREDIT',
       'NAME_YIELD_GROUP_mean_AMT_DOWN_PAYMENT',
       'NAME_YIELD_GROUP_mean_DAYS_LAST_DUE_1ST_VERSION',
       'NAME_YIELD_GROUP_mean_DAYS_TERMINATION',
       'NAME_YIELD_GROUP_mean_AMT_GOODS_PRICE',
       'NAME_YIELD_GROUP_mean_CNT_PAYMENT',
       'NAME_YIELD_GROUP_mean_DAYS_DECISION',
       'NAME_YIELD_GROUP_mean_HOUR_APPR_PROCESS_START',
       'NAME_YIELD_GROUP_mean_RATE_DOWN_PAYMENT',
       'PRODUCT_COMBINATION_mean_AMT_ANNUITY',
       'PRODUCT_COMBINATION_mean_AMT_APPLICATION',
       'PRODUCT_COMBINATION_mean_AMT_CREDIT',
       'PRODUCT_COMBINATION_mean_AMT_DOWN_PAYMENT',
       'PRODUCT_COMBINATION_mean_DAYS_LAST_DUE_1ST_VERSION',
       'PRODUCT_COMBINATION_mean_DAYS_TERMINATION',
       'PRODUCT_COMBINATION_mean_AMT_GOODS_PRICE',
       'PRODUCT_COMBINATION_mean_CNT_PAYMENT',
       'PRODUCT_COMBINATION_mean_DAYS_DECISION',
       'PRODUCT_COMBINATION_mean_HOUR_APPR_PROCESS_START',
       'PRODUCT_COMBINATION_mean_RATE_DOWN_PAYMENT'
                   ]:
        previous_application_agg_RECIPIES.append((select, agg))
previous_application_agg_RECIPIES = [(['SK_ID_CURR'], previous_application_agg_RECIPIES)]
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(previous_application_agg_RECIPIES):
    group_object = previous_application.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        all_data = all_data.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [35]:
previous_application.select_dtypes(include=['object']).columns.values


array(['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
       'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION'],
      dtype=object)

In [37]:
previous_application = previous_application[['SK_ID_CURR','NAME_CONTRACT_TYPE','FLAG_LAST_APPL_PER_CONTRACT', 'NAME_PRODUCT_TYPE','NAME_CASH_LOAN_PURPOSE','CODE_REJECT_REASON']]

In [38]:
previous_application = pd.get_dummies(previous_application)

In [38]:
previous_application.columns.values

array(['SK_ID_CURR', 'NAME_CONTRACT_TYPE_Cash loans',
       'NAME_CONTRACT_TYPE_Consumer loans',
       'NAME_CONTRACT_TYPE_Revolving loans', 'NAME_CONTRACT_TYPE_XNA',
       'FLAG_LAST_APPL_PER_CONTRACT_N', 'FLAG_LAST_APPL_PER_CONTRACT_Y',
       'NAME_PRODUCT_TYPE_XNA', 'NAME_PRODUCT_TYPE_walk-in',
       'NAME_PRODUCT_TYPE_x-sell',
       'NAME_CASH_LOAN_PURPOSE_Building a house or an annex',
       'NAME_CASH_LOAN_PURPOSE_Business development',
       'NAME_CASH_LOAN_PURPOSE_Buying a garage',
       'NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land',
       'NAME_CASH_LOAN_PURPOSE_Buying a home',
       'NAME_CASH_LOAN_PURPOSE_Buying a new car',
       'NAME_CASH_LOAN_PURPOSE_Buying a used car',
       'NAME_CASH_LOAN_PURPOSE_Car repairs',
       'NAME_CASH_LOAN_PURPOSE_Education',
       'NAME_CASH_LOAN_PURPOSE_Everyday expenses',
       'NAME_CASH_LOAN_PURPOSE_Furniture',
       'NAME_CASH_LOAN_PURPOSE_Gasification / water supply',
       'NAME_CASH_LOAN_PURPOSE_Hobby', 'NAME

In [39]:
previous_application_agg_RECIPIES = []
for agg in [ 'mean','min', 'max', 'sum', 'var']:
    for select in ['NAME_CONTRACT_TYPE_Cash loans',
       'NAME_CONTRACT_TYPE_Consumer loans',
       'NAME_CONTRACT_TYPE_Revolving loans', 'NAME_CONTRACT_TYPE_XNA',
       'FLAG_LAST_APPL_PER_CONTRACT_N', 'FLAG_LAST_APPL_PER_CONTRACT_Y',
       'NAME_PRODUCT_TYPE_XNA', 'NAME_PRODUCT_TYPE_walk-in',
       'NAME_PRODUCT_TYPE_x-sell',
       'NAME_CASH_LOAN_PURPOSE_Building a house or an annex',
       'NAME_CASH_LOAN_PURPOSE_Business development',
       'NAME_CASH_LOAN_PURPOSE_Buying a garage',
       'NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land',
       'NAME_CASH_LOAN_PURPOSE_Buying a home',
       'NAME_CASH_LOAN_PURPOSE_Buying a new car',
       'NAME_CASH_LOAN_PURPOSE_Buying a used car',
       'NAME_CASH_LOAN_PURPOSE_Car repairs',
       'NAME_CASH_LOAN_PURPOSE_Education',
       'NAME_CASH_LOAN_PURPOSE_Everyday expenses',
       'NAME_CASH_LOAN_PURPOSE_Furniture',
       'NAME_CASH_LOAN_PURPOSE_Gasification / water supply',
       'NAME_CASH_LOAN_PURPOSE_Hobby', 'NAME_CASH_LOAN_PURPOSE_Journey',
       'NAME_CASH_LOAN_PURPOSE_Medicine',
       'NAME_CASH_LOAN_PURPOSE_Money for a third person',
       'NAME_CASH_LOAN_PURPOSE_Other',
       'NAME_CASH_LOAN_PURPOSE_Payments on other loans',
       'NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment',
       'NAME_CASH_LOAN_PURPOSE_Refusal to name the goal',
       'NAME_CASH_LOAN_PURPOSE_Repairs',
       'NAME_CASH_LOAN_PURPOSE_Urgent needs',
       'NAME_CASH_LOAN_PURPOSE_Wedding / gift / holiday',
       'NAME_CASH_LOAN_PURPOSE_XAP', 'NAME_CASH_LOAN_PURPOSE_XNA',
       'CODE_REJECT_REASON_CLIENT', 'CODE_REJECT_REASON_HC',
       'CODE_REJECT_REASON_LIMIT', 'CODE_REJECT_REASON_SCO',
       'CODE_REJECT_REASON_SCOFR', 'CODE_REJECT_REASON_SYSTEM',
       'CODE_REJECT_REASON_VERIF', 'CODE_REJECT_REASON_XAP',
       'CODE_REJECT_REASON_XNA'
                   ]:
        previous_application_agg_RECIPIES.append((select, agg))
previous_application_agg_RECIPIES = [(['SK_ID_CURR'], previous_application_agg_RECIPIES)]
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(previous_application_agg_RECIPIES):
    group_object = previous_application.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        all_data = all_data.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [40]:
del previous_application

In [41]:
bureau.shape

(1716428, 18)

In [42]:
bureau_balance.shape
bureau_balance = pd.get_dummies(bureau_balance)
bureau_balance_agg = bureau_balance.groupby('SK_ID_BUREAU', as_index = False).agg([ 'mean', 'sum']).reset_index()
bureau_balance_agg.head()
# List of column names
columns = ['SK_ID_BUREAU']

# Iterate through the variables names
for var in bureau_balance_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_BUREAU':
        
        # Iterate through the stat names
        for stat in bureau_balance_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('bureau_balance_%s_%s' % (var, stat))
bureau_balance_agg.columns = columns
bureau_balance_agg.head 

<bound method DataFrame.head of         SK_ID_BUREAU  bureau_balance_MONTHS_BALANCE_mean  \
0            5001709                               -48.0   
1            5001710                               -41.0   
2            5001711                                -1.5   
3            5001712                                -9.0   
4            5001713                               -10.5   
5            5001714                                -7.0   
6            5001715                               -29.5   
7            5001716                               -42.5   
8            5001717                               -10.5   
9            5001718                               -19.0   
10           5001719                               -21.0   
11           5001720                               -17.5   
12           5001721                               -42.0   
13           5001722                               -42.5   
14           5001723                               -15.0   
15      

In [43]:
bureau = bureau.merge(bureau_balance_agg, on = 'SK_ID_BUREAU', how = 'left')

In [45]:
bureau.columns.values

array(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE',
       'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT',
       'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE',
       'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY', 'DAYS_DIFF',
       'bureau_balance_MONTHS_BALANCE_mean',
       'bureau_balance_MONTHS_BALANCE_sum',
       'bureau_balance_STATUS_0_mean', 'bureau_balance_STATUS_0_sum',
       'bureau_balance_STATUS_1_mean', 'bureau_balance_STATUS_1_sum',
       'bureau_balance_STATUS_2_mean', 'bureau_balance_STATUS_2_sum',
       'bureau_balance_STATUS_3_mean', 'bureau_balance_STATUS_3_sum',
       'bureau_balance_STATUS_4_mean', 'bureau_balance_STATUS_4_sum',
       'bureau_balance_STATUS_5_mean', 'bureau_balance_STATUS_5_sum',
       'bureau_balance_STATUS_C_mean', 'bureau_balance_STATUS_C_sum',
       'bureau_balance_STATU

In [46]:
bureau.select_dtypes(exclude=['object']).columns.values

array(['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE',
       'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT',
       'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM',
       'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY',
       'DAYS_DIFF', 'bureau_balance_MONTHS_BALANCE_mean',
       'bureau_balance_MONTHS_BALANCE_sum',
       'bureau_balance_STATUS_0_mean', 'bureau_balance_STATUS_0_sum',
       'bureau_balance_STATUS_1_mean', 'bureau_balance_STATUS_1_sum',
       'bureau_balance_STATUS_2_mean', 'bureau_balance_STATUS_2_sum',
       'bureau_balance_STATUS_3_mean', 'bureau_balance_STATUS_3_sum',
       'bureau_balance_STATUS_4_mean', 'bureau_balance_STATUS_4_sum',
       'bureau_balance_STATUS_5_mean', 'bureau_balance_STATUS_5_sum',
       'bureau_balance_STATUS_C_mean', 'bureau_balance_STATUS_C_sum',
       'bureau_balance_STATUS_X_mean', 'bureau_balance_STATUS_X_sum'],
      dtype=obj

In [44]:
AGGREGATION_RECIPIES = [

    (['CREDIT_TYPE'], [('AMT_ANNUITY', 'mean'),
                           ('CREDIT_DAY_OVERDUE', 'mean'),
                           ('AMT_CREDIT_MAX_OVERDUE', 'mean'),
                           ('DAYS_CREDIT_ENDDATE', 'mean'),
                           ('AMT_CREDIT_SUM_OVERDUE', 'mean'),
                       ('bureau_balance_MONTHS_BALANCE_sum', 'mean'),
                           ('AMT_CREDIT_SUM_LIMIT', 'mean')]),
        (['CREDIT_ACTIVE'], [('AMT_ANNUITY', 'mean'),
                           ('CREDIT_DAY_OVERDUE', 'mean'),
                             ('bureau_balance_MONTHS_BALANCE_sum', 'mean'),
                           ('AMT_CREDIT_MAX_OVERDUE', 'mean'),
                           ('DAYS_CREDIT_ENDDATE', 'mean'),
                           ('AMT_CREDIT_SUM_OVERDUE', 'mean'),
                           ('AMT_CREDIT_SUM_LIMIT', 'mean')]),
]
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(AGGREGATION_RECIPIES):
    group_object = bureau.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        bureau = bureau.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [45]:
bureau = bureau.drop(columns = ['CREDIT_CURRENCY','CREDIT_TYPE','SK_ID_BUREAU'] )
bureau = pd.get_dummies(bureau)

In [46]:
bureau_agg = bureau.groupby('SK_ID_CURR', as_index = False).agg([ 'mean', 'max', 'min', 'sum']).reset_index()
bureau_agg.head()
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in bureau_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in bureau_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('bureau_%s_%s' % (var, stat))
bureau_agg.columns = columns
bureau_agg.columns.values 

array(['SK_ID_CURR', 'bureau_DAYS_CREDIT_mean', 'bureau_DAYS_CREDIT_max',
       'bureau_DAYS_CREDIT_min', 'bureau_DAYS_CREDIT_sum',
       'bureau_CREDIT_DAY_OVERDUE_mean', 'bureau_CREDIT_DAY_OVERDUE_max',
       'bureau_CREDIT_DAY_OVERDUE_min', 'bureau_CREDIT_DAY_OVERDUE_sum',
       'bureau_DAYS_CREDIT_ENDDATE_mean',
       'bureau_DAYS_CREDIT_ENDDATE_max', 'bureau_DAYS_CREDIT_ENDDATE_min',
       'bureau_DAYS_CREDIT_ENDDATE_sum', 'bureau_DAYS_ENDDATE_FACT_mean',
       'bureau_DAYS_ENDDATE_FACT_max', 'bureau_DAYS_ENDDATE_FACT_min',
       'bureau_DAYS_ENDDATE_FACT_sum',
       'bureau_AMT_CREDIT_MAX_OVERDUE_mean',
       'bureau_AMT_CREDIT_MAX_OVERDUE_max',
       'bureau_AMT_CREDIT_MAX_OVERDUE_min',
       'bureau_AMT_CREDIT_MAX_OVERDUE_sum',
       'bureau_CNT_CREDIT_PROLONG_mean', 'bureau_CNT_CREDIT_PROLONG_max',
       'bureau_CNT_CREDIT_PROLONG_min', 'bureau_CNT_CREDIT_PROLONG_sum',
       'bureau_AMT_CREDIT_SUM_mean', 'bureau_AMT_CREDIT_SUM_max',
       'bureau_AMT_CREDIT_SUM

In [47]:
all_data = all_data.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')

In [48]:
all_data.shape

(356255, 1082)

In [49]:
del bureau_agg
del bureau
del bureau_balance_agg
del bureau_balance

In [50]:
POS_CASH_balance.select_dtypes(exclude=['object']).columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'SK_DPD', 'SK_DPD_DEF',
       'pos_cash_paid_late', 'pos_cash_paid_late_with_tolerance'],
      dtype=object)

In [51]:
POS_CASH_balance.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF', 'pos_cash_paid_late',
       'pos_cash_paid_late_with_tolerance'], dtype=object)

In [52]:
AGGREGATION_RECIPIES = [

    (['NAME_CONTRACT_STATUS'], [('CNT_INSTALMENT', 'mean'),
                           ('CNT_INSTALMENT_FUTURE', 'mean'),
                           ('SK_DPD', 'mean'),
                           ('SK_DPD_DEF', 'mean')
                           ]),
]
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(AGGREGATION_RECIPIES):
    group_object = POS_CASH_balance.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        POS_CASH_balance = POS_CASH_balance.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [53]:
POS_CASH_balance.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF', 'pos_cash_paid_late',
       'pos_cash_paid_late_with_tolerance',
       'NAME_CONTRACT_STATUS_mean_CNT_INSTALMENT',
       'NAME_CONTRACT_STATUS_mean_CNT_INSTALMENT_FUTURE',
       'NAME_CONTRACT_STATUS_mean_SK_DPD',
       'NAME_CONTRACT_STATUS_mean_SK_DPD_DEF'], dtype=object)

In [54]:
POS_CASH_balance = pd.get_dummies(POS_CASH_balance)

POS_CASH_balance_agg = POS_CASH_balance.drop(columns = ['SK_ID_PREV']).groupby('SK_ID_CURR', as_index = False).agg([ 'mean', 'max', 'min', 'sum']).reset_index()
POS_CASH_balance_agg.head()
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in POS_CASH_balance_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in POS_CASH_balance_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('POS_CASH_%s_%s' % (var, stat))
POS_CASH_balance_agg.columns = columns
POS_CASH_balance_agg.columns.values

array(['SK_ID_CURR', 'POS_CASH_MONTHS_BALANCE_mean',
       'POS_CASH_MONTHS_BALANCE_max', 'POS_CASH_MONTHS_BALANCE_min',
       'POS_CASH_MONTHS_BALANCE_sum', 'POS_CASH_CNT_INSTALMENT_mean',
       'POS_CASH_CNT_INSTALMENT_max', 'POS_CASH_CNT_INSTALMENT_min',
       'POS_CASH_CNT_INSTALMENT_sum',
       'POS_CASH_CNT_INSTALMENT_FUTURE_mean',
       'POS_CASH_CNT_INSTALMENT_FUTURE_max',
       'POS_CASH_CNT_INSTALMENT_FUTURE_min',
       'POS_CASH_CNT_INSTALMENT_FUTURE_sum', 'POS_CASH_SK_DPD_mean',
       'POS_CASH_SK_DPD_max', 'POS_CASH_SK_DPD_min',
       'POS_CASH_SK_DPD_sum', 'POS_CASH_SK_DPD_DEF_mean',
       'POS_CASH_SK_DPD_DEF_max', 'POS_CASH_SK_DPD_DEF_min',
       'POS_CASH_SK_DPD_DEF_sum', 'POS_CASH_pos_cash_paid_late_mean',
       'POS_CASH_pos_cash_paid_late_max',
       'POS_CASH_pos_cash_paid_late_min',
       'POS_CASH_pos_cash_paid_late_sum',
       'POS_CASH_pos_cash_paid_late_with_tolerance_mean',
       'POS_CASH_pos_cash_paid_late_with_tolerance_max',
       'POS_C

In [55]:
all_data= all_data.merge(POS_CASH_balance_agg, on = 'SK_ID_CURR', how = 'left')

In [54]:
all_data.shape

(356255, 1159)

In [56]:
del POS_CASH_balance_agg
del POS_CASH_balance

In [60]:
installments_payments.select_dtypes(exclude=['object']).columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT', 'instalment_paid_late_in_days',
       'instalment_paid_late', 'instalment_paid_over_amount',
       'instalment_paid_over'], dtype=object)

In [57]:


installments_payments['PAYMENT_PERC'] = installments_payments['AMT_PAYMENT'] / installments_payments['AMT_INSTALMENT']


installments_payments['DBD'] = installments_payments['DAYS_INSTALMENT'] - installments_payments['DAYS_ENTRY_PAYMENT']
installments_payments['instalment_paid_late_in_days'] = installments_payments['instalment_paid_late_in_days'].apply(lambda x: x if x > 0 else 0)
installments_payments['DBD'] = installments_payments['DBD'].apply(lambda x: x if x > 0 else 0)

In [58]:
installments_payments_agg = installments_payments.drop(columns = ['SK_ID_PREV']).groupby('SK_ID_CURR', as_index = False).agg([ 'mean', 'max', 'min', 'sum','std']).reset_index()
installments_payments_agg.head()
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in installments_payments_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in installments_payments_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('install_%s_%s' % (var, stat))
installments_payments_agg.columns = columns
installments_payments_agg.columns.values

array(['SK_ID_CURR', 'install_NUM_INSTALMENT_VERSION_mean',
       'install_NUM_INSTALMENT_VERSION_max',
       'install_NUM_INSTALMENT_VERSION_min',
       'install_NUM_INSTALMENT_VERSION_sum',
       'install_NUM_INSTALMENT_VERSION_std',
       'install_NUM_INSTALMENT_NUMBER_mean',
       'install_NUM_INSTALMENT_NUMBER_max',
       'install_NUM_INSTALMENT_NUMBER_min',
       'install_NUM_INSTALMENT_NUMBER_sum',
       'install_NUM_INSTALMENT_NUMBER_std',
       'install_DAYS_INSTALMENT_mean', 'install_DAYS_INSTALMENT_max',
       'install_DAYS_INSTALMENT_min', 'install_DAYS_INSTALMENT_sum',
       'install_DAYS_INSTALMENT_std', 'install_DAYS_ENTRY_PAYMENT_mean',
       'install_DAYS_ENTRY_PAYMENT_max', 'install_DAYS_ENTRY_PAYMENT_min',
       'install_DAYS_ENTRY_PAYMENT_sum', 'install_DAYS_ENTRY_PAYMENT_std',
       'install_AMT_INSTALMENT_mean', 'install_AMT_INSTALMENT_max',
       'install_AMT_INSTALMENT_min', 'install_AMT_INSTALMENT_sum',
       'install_AMT_INSTALMENT_std', 'inst

In [59]:
all_data= all_data.merge(installments_payments_agg, on = 'SK_ID_CURR', how = 'left')

In [60]:
all_data.shape

(356255, 1222)

In [61]:
del installments_payments_agg
del installments_payments

In [62]:
credit_card_balance.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE',
       'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT',
       'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT',
       'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM',
       'NAME_CONTRACT_STATUS', 'SK_DPD', 'SK_DPD_DEF'], dtype=object)

In [63]:
credit_card_balance.select_dtypes(exclude=['object']).columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE',
       'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT',
       'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT',
       'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 'SK_DPD',
       'SK_DPD_DEF'], dtype=object)

In [64]:
AGGREGATION_RECIPIES = [

    (['NAME_CONTRACT_STATUS'], [('AMT_BALANCE', 'mean'),
                           ('AMT_PAYMENT_TOTAL_CURRENT', 'mean'),
                           ('AMT_PAYMENT_CURRENT', 'mean'),
                           ('CNT_INSTALMENT_MATURE_CUM', 'mean'),
                           ('SK_DPD_DEF', 'mean'),
                           ('SK_DPD', 'mean'),
                           ('AMT_CREDIT_LIMIT_ACTUAL', 'mean')]),
]
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(AGGREGATION_RECIPIES):
    group_object = credit_card_balance.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        credit_card_balance = credit_card_balance.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [29]:
credit_card_balance.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE',
       'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT',
       'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT',
       'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM',
       'NAME_CONTRACT_STATUS', 'SK_DPD', 'SK_DPD_DEF',
       'NAME_CONTRACT_STATUS_mean_AMT_BALANCE',
       'NAME_CONTRACT_STATUS_mean_AMT_PAYMENT_TOTAL_CURRENT',
       'NAME_CONTRACT_STATUS_mean_AMT_PAYMENT_CURRENT',
       'NAME_CONTRACT_STATUS_mean_CNT_INSTALMENT_MATURE_CUM',
       'NAME_CONTRACT_STATUS_mean_SK_DPD_DEF',
       'NAME_CONTRACT_STATUS_mean_SK_DPD',
       'NAME_CONTRACT_STATUS_mean_AMT_CREDIT_LIMIT_ACTUAL'], dtype=object)

In [65]:
credit_card_balance = pd.get_dummies(credit_card_balance)
credit_card_balance_agg = credit_card_balance.drop(columns = ['SK_ID_PREV']).groupby('SK_ID_CURR', as_index = False).agg([ 'mean', 'max', 'min', 'sum','var']).reset_index()

# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in credit_card_balance_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in credit_card_balance_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('credit_card_%s_%s' % (var, stat))
credit_card_balance_agg.columns = columns
credit_card_balance_agg.columns.values

array(['SK_ID_CURR', 'credit_card_MONTHS_BALANCE_mean',
       'credit_card_MONTHS_BALANCE_max', 'credit_card_MONTHS_BALANCE_min',
       'credit_card_MONTHS_BALANCE_sum', 'credit_card_MONTHS_BALANCE_var',
       'credit_card_AMT_BALANCE_mean', 'credit_card_AMT_BALANCE_max',
       'credit_card_AMT_BALANCE_min', 'credit_card_AMT_BALANCE_sum',
       'credit_card_AMT_BALANCE_var',
       'credit_card_AMT_CREDIT_LIMIT_ACTUAL_mean',
       'credit_card_AMT_CREDIT_LIMIT_ACTUAL_max',
       'credit_card_AMT_CREDIT_LIMIT_ACTUAL_min',
       'credit_card_AMT_CREDIT_LIMIT_ACTUAL_sum',
       'credit_card_AMT_CREDIT_LIMIT_ACTUAL_var',
       'credit_card_AMT_DRAWINGS_ATM_CURRENT_mean',
       'credit_card_AMT_DRAWINGS_ATM_CURRENT_max',
       'credit_card_AMT_DRAWINGS_ATM_CURRENT_min',
       'credit_card_AMT_DRAWINGS_ATM_CURRENT_sum',
       'credit_card_AMT_DRAWINGS_ATM_CURRENT_var',
       'credit_card_AMT_DRAWINGS_CURRENT_mean',
       'credit_card_AMT_DRAWINGS_CURRENT_max',
       'credit_

In [66]:
credit_card_balance_agg = credit_card_balance_agg.drop(columns = [
       'credit_card_NAME_CONTRACT_STATUS_Active_min',

       'credit_card_NAME_CONTRACT_STATUS_Active_var',

       'credit_card_NAME_CONTRACT_STATUS_Approved_min',

       'credit_card_NAME_CONTRACT_STATUS_Approved_var',

       'credit_card_NAME_CONTRACT_STATUS_Completed_min',

       'credit_card_NAME_CONTRACT_STATUS_Completed_var',

       'credit_card_NAME_CONTRACT_STATUS_Demand_min',


       'credit_card_NAME_CONTRACT_STATUS_Refused_min',

       'credit_card_NAME_CONTRACT_STATUS_Refused_var',

       'credit_card_NAME_CONTRACT_STATUS_Sent proposal_min',
  
       'credit_card_NAME_CONTRACT_STATUS_Sent proposal_var',

       'credit_card_NAME_CONTRACT_STATUS_Signed_min',

       'credit_card_NAME_CONTRACT_STATUS_Signed_var']) 

In [67]:
all_data_try = all_data
all_data_try = all_data_try.merge(credit_card_balance_agg, on = 'SK_ID_CURR', how = 'left')
all_data_try.shape

(356255, 1379)

In [68]:
del all_data

In [71]:
del credit_card_balance_agg
del credit_card_balance

In [73]:
all_data_try.shape

(356255, 1379)

In [70]:
all_data_try.to_csv('all_data_try_v5.csv')
#all_data_try = pd.read_csv('all_data_try_v2.csv')

In [2]:
all_data_try = pd.read_csv('all_data_try_v5.csv')

In [3]:
all_data_try= pd.get_dummies(all_data_try)

In [6]:
all_data_try = all_data_try.drop(columns =['Unnamed: 0'])

In [5]:
corrs = all_data_try.corr()

In [6]:

threshold = 0.985

# Empty dictionary to hold correlated variables
above_threshold_vars = {}

# For each column, record the variables that are above the threshold
for col in corrs:
    above_threshold_vars[col] = list(corrs.index[corrs[col] > threshold])
    cols_to_remove = []
cols_seen = []
cols_to_remove_pair = []

# Iterate through columns and correlated columns
for key, value in above_threshold_vars.items():
    # Keep track of columns already examined
    cols_seen.append(key)
    for x in value:
        if x == key:
            next
        else:
            # Only want to remove one in a pair
            if x not in cols_seen:
                cols_to_remove.append(x)
                cols_to_remove_pair.append(key)
            
cols_to_remove = list(set(cols_to_remove))
print('Number of columns to remove: ', len(cols_to_remove))

('Number of columns to remove: ', 325)


In [8]:
for col in cols_to_remove:
    del all_data_try[col]

In [7]:
all_data_try.shape

(356255, 1500)

In [8]:
ntrain = 307511
ntest = 48744
test_ID = all_data_try[ntrain:]['SK_ID_CURR']


#all_data_try = all_data_try.astype(float)
train_data = all_data_try[:ntrain]
test_data = all_data_try[ntrain:]
del test_data['TARGET']
y__train = train_data.TARGET.values
del train_data['TARGET']

In [9]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [28]:
train_data['NEW_DOC_IND_KURT'].max()


20.00000000000001

In [27]:
train_data['NEW_INC_BY_ORG'].max()

225000.0

In [10]:
#y_train=application_train.TARGET.values
x_train, x_val, y_train, y_val = train_test_split(train_data, y__train, test_size=0.2, random_state=18)
lgb_train = lgb.Dataset(data=x_train, label=y_train)
lgb_eval = lgb.Dataset(data=x_val, label=y_val)


In [11]:
del x_train, x_val, y_train, y_val,all_data_try

In [12]:
import gc
gc.collect()

74

In [14]:
params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
          'learning_rate': 0.01, 'num_leaves': 32, 'num_iteration': 10000, 'verbose': -1 ,'silent':-1,
          'colsample_bytree':0.05, 'subsample':1, 'max_depth':8, 'reg_alpha':0.04, 'reg_lambda':0.073, 
          'min_split_gain':0.02, 'min_child_weight':40,'scale_pos_weight': 1}

model = lgb.train(params, lgb_train, valid_sets=lgb_eval,verbose_eval=200,early_stopping_rounds=300)

Training until validation scores don't improve for 300 rounds.
[200]	valid_0's auc: 0.766512
[400]	valid_0's auc: 0.77623
[600]	valid_0's auc: 0.781249
[800]	valid_0's auc: 0.784899
[1000]	valid_0's auc: 0.787266
[1200]	valid_0's auc: 0.78907
[1400]	valid_0's auc: 0.790284
[1600]	valid_0's auc: 0.791135
[1800]	valid_0's auc: 0.791758
[2000]	valid_0's auc: 0.792435
[2200]	valid_0's auc: 0.792931
[2400]	valid_0's auc: 0.793237
[2600]	valid_0's auc: 0.793484
[2800]	valid_0's auc: 0.793612
[3000]	valid_0's auc: 0.79386
[3200]	valid_0's auc: 0.793958
[3400]	valid_0's auc: 0.794048
[3600]	valid_0's auc: 0.794115
[3800]	valid_0's auc: 0.794227
[4000]	valid_0's auc: 0.794254
Early stopping, best iteration is:
[3869]	valid_0's auc: 0.794274


In [73]:
y_train=application_train.TARGET.values
x_train, x_val, y_train, y_val = train_test_split(train_data, y_train, test_size=0.2, random_state=18)
lgb_train = lgb.Dataset(data=x_train, label=y_train)
lgb_eval = lgb.Dataset(data=x_val, label=y_val)
params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
          'learning_rate': 0.01,'num_leaves': 32, 'num_iteration': 10000, 'verbose': -1 ,'silent':-1,
          'colsample_bytree':0.05, 'subsample':0.5, 'max_depth':8, 'reg_alpha':0, 'reg_lambda':100, 
          'min_split_gain':0.5, 'min_child_weight':40,'scale_pos_weight': 1}
model = lgb.train(params, lgb_train, valid_sets=lgb_eval,verbose_eval=200,early_stopping_rounds=300)

Training until validation scores don't improve for 300 rounds.
[200]	valid_0's auc: 0.764339
[400]	valid_0's auc: 0.772416
[600]	valid_0's auc: 0.777903
[800]	valid_0's auc: 0.781338
[1000]	valid_0's auc: 0.784136
[1200]	valid_0's auc: 0.786093
[1400]	valid_0's auc: 0.787703
[1600]	valid_0's auc: 0.788912
[1800]	valid_0's auc: 0.789965
[2000]	valid_0's auc: 0.790784
[2200]	valid_0's auc: 0.79143
[2400]	valid_0's auc: 0.791957
[2600]	valid_0's auc: 0.792492
[2800]	valid_0's auc: 0.792812
[3000]	valid_0's auc: 0.793091
[3200]	valid_0's auc: 0.793318
[3400]	valid_0's auc: 0.793491
[3600]	valid_0's auc: 0.793727
[3800]	valid_0's auc: 0.793815
[4000]	valid_0's auc: 0.793898
[4200]	valid_0's auc: 0.793998
[4400]	valid_0's auc: 0.794073
[4600]	valid_0's auc: 0.794239
[4800]	valid_0's auc: 0.794293
[5000]	valid_0's auc: 0.794285
[5200]	valid_0's auc: 0.794343
[5400]	valid_0's auc: 0.794359
[5600]	valid_0's auc: 0.794388
[5800]	valid_0's auc: 0.794396
[6000]	valid_0's auc: 0.794394
Early stoppi

In [19]:

lgb_train = lgb.Dataset(data=train_data, label=y__train)
params ={'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
          'learning_rate': 0.01, 'num_leaves': 32, 'num_iteration': 10000, 'verbose': -1 ,'silent':-1,
          'colsample_bytree':0.1, 'subsample':0.5, 'max_depth':8, 'reg_alpha':0, 'reg_lambda':100, 
          'min_split_gain':0.2, 'min_child_weight':40,'scale_pos_weight': 1}
model = lgb.cv(params, lgb_train,verbose_eval=200,early_stopping_rounds=300)

[200]	cv_agg's auc: 0.762927 + 0.00433614
[400]	cv_agg's auc: 0.772544 + 0.00388293
[600]	cv_agg's auc: 0.778535 + 0.00345367
[800]	cv_agg's auc: 0.782512 + 0.0033181
[1000]	cv_agg's auc: 0.785235 + 0.00321738
[1200]	cv_agg's auc: 0.787099 + 0.00309384
[1400]	cv_agg's auc: 0.788518 + 0.00302637
[1600]	cv_agg's auc: 0.789696 + 0.00294613
[1800]	cv_agg's auc: 0.790536 + 0.00290679
[2000]	cv_agg's auc: 0.79123 + 0.00290265
[2200]	cv_agg's auc: 0.791771 + 0.00289328
[2400]	cv_agg's auc: 0.792221 + 0.00288357
[2600]	cv_agg's auc: 0.792609 + 0.00287585
[2800]	cv_agg's auc: 0.792928 + 0.00288487
[3000]	cv_agg's auc: 0.793232 + 0.00287129
[3200]	cv_agg's auc: 0.793462 + 0.00285051
[3400]	cv_agg's auc: 0.793658 + 0.00285222
[3600]	cv_agg's auc: 0.793812 + 0.00285635
[3800]	cv_agg's auc: 0.793949 + 0.0028537
[4000]	cv_agg's auc: 0.79403 + 0.002892
[4200]	cv_agg's auc: 0.794104 + 0.0028686
[4400]	cv_agg's auc: 0.794164 + 0.00289012
[4600]	cv_agg's auc: 0.794225 + 0.00289657
[4800]	cv_agg's auc: 0

In [21]:
#y_train=application_train.TARGET.values
lgb_train = lgb.Dataset(data=train_data, label=y__train)
params ={'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
          'learning_rate': 0.01, 'num_leaves': 32, 'num_iteration': 5700, 'verbose': -1 ,'silent':-1,
          'colsample_bytree':0.1, 'subsample':0.5, 'max_depth':8, 'reg_alpha':0, 'reg_lambda':100, 
          'min_split_gain':0.2, 'min_child_weight':40,'scale_pos_weight': 1}
model = lgb.train(params, lgb_train)
preds = model.predict(test_data.values)
sub_lgb = pd.DataFrame()
sub_lgb['SK_ID_CURR'] = test_ID
sub_lgb['TARGET'] = preds
sub_lgb.to_csv("lgb_v10_5700.csv", index=False)
sub_lgb.head()

,SK_ID_CURR,TARGET
307511,100001,0.035204
307512,100005,0.180012
307513,100013,0.023314
307514,100028,0.052763
307515,100038,0.119242


In [89]:
del lgb_train

In [ ]:
sub_lgb.head(50)

In [32]:
sub_lgb['SK_ID_CURR'] = application_test['SK_ID_CURR']
sub_lgb['TARGET'] = preds
sub_lgb.to_csv("lgb_clean_3400.csv", index=False)
sub_lgb.head()

,SK_ID_CURR,TARGET
0,100001,0.041611
1,100005,0.140239
2,100013,0.023746
3,100028,0.045194
4,100038,0.129658


In [31]:
application_test = pd.read_csv('application_test.csv')

In [13]:
sub_lgb.to_csv("lgb_baseline_3200.csv", index=False)

In [5]:
import xgboost as xgb

In [6]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

In [10]:

#y_train=application_train.TARGET.values
x_train, x_val, y_train, y_val = train_test_split(train_data, y__train, test_size=0.2, random_state=18)
model=XGBClassifier(learning_rate =0.01, n_estimators=10000, max_depth=8, min_child_weight=40, subsample=1, colsample_bytree=0.05, objective= 'binary:logistic', 
            nthread=-1, scale_pos_weight=1, reg_lambda = 100)
  
eval_set=[(x_val,y_val)]

In [11]:

model.fit(x_train,y_train,early_stopping_rounds=300,eval_metric="auc",eval_set=eval_set,verbose=100)

[0]	validation_0-auc:0.597057
Will train until validation_0-auc hasn't improved in 300 rounds.
[100]	validation_0-auc:0.743774
[200]	validation_0-auc:0.755536
[300]	validation_0-auc:0.763625
[400]	validation_0-auc:0.768701
[500]	validation_0-auc:0.773178
[600]	validation_0-auc:0.776368
[700]	validation_0-auc:0.779031
[800]	validation_0-auc:0.781118
[900]	validation_0-auc:0.782877
[1000]	validation_0-auc:0.78433
[1100]	validation_0-auc:0.78573
[1200]	validation_0-auc:0.786816
[1300]	validation_0-auc:0.787688
[1400]	validation_0-auc:0.788457
[1500]	validation_0-auc:0.789178
[1600]	validation_0-auc:0.789764
[1700]	validation_0-auc:0.790266
[1800]	validation_0-auc:0.790777
[1900]	validation_0-auc:0.79128
[2000]	validation_0-auc:0.791671
[2100]	validation_0-auc:0.791983
[2200]	validation_0-auc:0.792283
[2300]	validation_0-auc:0.792605
[2400]	validation_0-auc:0.792817
[2500]	validation_0-auc:0.793003
[2600]	validation_0-auc:0.793202
[2700]	validation_0-auc:0.793429
[2800]	validation_0-auc:0.

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.05, gamma=0, learning_rate=0.01,
       max_delta_step=0, max_depth=8, min_child_weight=40, missing=None,
       n_estimators=10000, n_jobs=1, nthread=-1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=100, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [12]:
import xgboost as xgb

dtrain = xgb.DMatrix(train_data,y__train)


# specify parameters via map, definition are same as c++ version
param = {'scale_pos_weight':1,'max_depth':8, 'eta':0.01, 'lambda' :100,'silent':1,'min_child_weight':40,'subsample':1, 'objective':'binary:logistic','colsample_bytree':0.05,}


num_round = 4838
model = xgb.train(param, dtrain, num_round)

In [13]:
dtest = xgb.DMatrix(test_data)
preds = model.predict(dtest)
sub_xgb = pd.DataFrame()
sub_xgb['SK_ID_CURR'] = test_ID
sub_xgb['TARGET'] = preds
sub_xgb.to_csv("xgb_v5_4838.csv", index=False)
sub_xgb.head()

,SK_ID_CURR,TARGET
307511,100001,0.042876
307512,100005,0.137632
307513,100013,0.019636
307514,100028,0.048132
307515,100038,0.123605
